In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

frequencySelectionApplied = 0    # 0 = do not apply, 1 = do apply

data = pd.read_csv("data/synthetic_data_lung_cancer.csv")

if frequencySelectionApplied == 1:
    data = pd.read_csv("data/bestFrequencyFiltered_t-low-0.3_t-high-1.csv")

print(data.columns)
data.tail()

Index(['SUBJECT_ID', 'DEFINITION_ID', 'TIME'], dtype='object')


,SUBJECT_ID,DEFINITION_ID,TIME
560966,984,measurement_1141,0.027321
560967,984,observation_156,0.028739
560968,984,measurement_1140,0.030802
560969,984,measurement_1327,0.035081
560970,984,condition_459,0.038022


In [76]:
data.loc[data['DEFINITION_ID'].str.contains('condition_*')]

#all condition observations are removed due to frequency Filtering

,SUBJECT_ID,DEFINITION_ID,TIME
1,1,condition_1922,0.008643
2,1,condition_785,0.027792
5,1,condition_1561,0.057292
7,1,condition_2391,0.065500
8,1,condition_175,0.166536
...,...,...,...
560922,984,condition_747,0.006456
560923,984,condition_1767,0.007445
560931,984,condition_517,0.008754
560964,984,condition_1496,0.023802


In [77]:
dataFormat = 1 # 0 for seperate ID column 1 for not
df = data.copy()
if dataFormat == 0:
    label_encoder = LabelEncoder()
    df['DEFINITION_ID_encoded'] = label_encoder.fit_transform(df['DEFINITION_ID'])
    df['condition'] = df['DEFINITION_ID'].apply(lambda x: 'condition' in x)
    df['procedure'] = df['DEFINITION_ID'].apply(lambda x: 'procedure' in x)
    df['drug'] = df['DEFINITION_ID'].apply(lambda x: 'drug' in x)
    df['observation'] = df['DEFINITION_ID'].apply(lambda x: 'observation' in x)
    df['measurement'] = df['DEFINITION_ID'].apply(lambda x: 'measurement' in x)
    df.drop(columns=['DEFINITION_ID'], inplace=True)
    df['time_since_last'] = df.groupby('SUBJECT_ID')['TIME'].diff().fillna(0)
    columns_to_convert = ['condition', 'procedure', 'drug', 'observation', 'measurement']
    for column in columns_to_convert:
        df[column] = df[column].astype(int)
    time_threshold = 1 
    df['DEATH'] = df['time_since_last'].apply(lambda x: 1 if x <= time_threshold else 0)
elif dataFormat == 1:
    # Splitting 'DEFINITION_ID' into multiple columns
    new_cols = df['DEFINITION_ID'].str.split('_', expand=True)
    # Naming the new columns
    new_cols.columns = ['CATEGORY', 'CATEGORY_ID']
    print("New unique categorie: "+new_cols['CATEGORY'].unique())
    # Pivoting the DataFrame
    pivot_new_cols = new_cols.pivot_table(index=data.index, columns='CATEGORY', values='CATEGORY_ID', aggfunc=lambda x: ', '.join(str(v) for v in x)).fillna('-1')
    # Concatenating the new columns to the original DataFrame
    df = pd.concat([df, pivot_new_cols], axis=1)
    # Dropping the original 'DEFINITION_ID' column
    df.drop(columns=['DEFINITION_ID'], inplace=True)
    # Filling the death column 
    df['death'] = df['death'].replace('None', '1')
    df['death'] = df['death'].replace('-1', '0')
    # Converting column to boolean dtype
    df['death'] = df['death'].astype(int)
    # Creating additional column
    df['time_since_last'] = df.groupby('SUBJECT_ID')['TIME'].diff().fillna(0)
else:
    ...

print(df.columns)
#df.loc[(df['condition']=='')&(df['drug']=='')&(df['measurement']=='')&(df['observation']=='')&(df['procedure']=='')]
[print("Unique values in feature "+x+": "+str(len(df[x].unique()))) for x in df.columns]

df.head()

['New unique categorie: drug' 'New unique categorie: condition'
 'New unique categorie: measurement' 'New unique categorie: observation'
 'New unique categorie: procedure' 'New unique categorie: death']
Index(['SUBJECT_ID', 'TIME', 'condition', 'death', 'drug', 'measurement',
       'observation', 'procedure', 'time_since_last'],
      dtype='object')
Unique values in feature SUBJECT_ID: 727
Unique values in feature TIME: 560956
Unique values in feature condition: 2400
Unique values in feature death: 2
Unique values in feature drug: 419
Unique values in feature measurement: 1333
Unique values in feature observation: 225
Unique values in feature procedure: 491
Unique values in feature time_since_last: 560198


,SUBJECT_ID,TIME,condition,death,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,0,217,-1,-1,-1,0.000000
1,1,0.008643,1922,0,-1,-1,-1,-1,0.003836
2,1,0.027792,785,0,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,0,49,-1,-1,-1,0.004722
4,1,0.056765,-1,0,-1,132,-1,-1,0.024250


In [78]:
#dummies for categorical analysis

createDummies = 0 # 0 = no, 1 = yes

if createDummies ==1 and dataFormat==1:
    df = pd.get_dummies(df,columns=['condition','drug','measurement','observation','procedure'])
    
df.head()

,SUBJECT_ID,TIME,condition,death,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,0,217,-1,-1,-1,0.000000
1,1,0.008643,1922,0,-1,-1,-1,-1,0.003836
2,1,0.027792,785,0,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,0,49,-1,-1,-1,0.004722
4,1,0.056765,-1,0,-1,132,-1,-1,0.024250


In [79]:
from sklearn.model_selection import train_test_split

# List of unique SUBJECT_IDs
unique_subject_ids = df['SUBJECT_ID'].unique()

# Split the unique_subject_ids into train and test IDs
train_ids, test_ids = train_test_split(unique_subject_ids, test_size=0.3, random_state=2)

# Filter the main DataFrame into train and test sets based on the selected IDs
train_data = df[df['SUBJECT_ID'].isin(train_ids)]
test_data = df[df['SUBJECT_ID'].isin(test_ids)]

# Separate the features (X) and target variable (y) for train and test sets
X_train = train_data.drop('death',axis=1)# Drop 'SUBJECT_ID' and target column
y_train = pd.Series(train_data.death)

X_test = test_data.drop('death',axis=1)# Drop 'SUBJECT_ID' and target column
y_test = pd.Series(test_data.death)

if createDummies ==1 and dataFormat==1:
    X_train = X_train.drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
    X_test = X_test.drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
#old way:
#X = df.drop('death',axis=1).drop(['condition_-1','drug_-1','measurement_-1','observation_-1','procedure_-1'],axis=1)
#y = pd.Series(df.death)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

print(y_train.head())
X_train.head()

0    0
1    0
2    0
3    0
4    0
Name: death, dtype: int32


,SUBJECT_ID,TIME,condition,drug,measurement,observation,procedure,time_since_last
0,1,0.004807,-1,217,-1,-1,-1,0.000000
1,1,0.008643,1922,-1,-1,-1,-1,0.003836
2,1,0.027792,785,-1,-1,-1,-1,0.019149
3,1,0.032515,-1,49,-1,-1,-1,0.004722
4,1,0.056765,-1,-1,132,-1,-1,0.024250


In [81]:
#undersampling

from sklearn.utils import resample

useUnderSampeling = 1 # 0 = no, 1 = yes

if useUnderSampeling == 1:
    # Separate majority and minority classes
    df_majority = df[df.death==0]
    df_minority = df[df.death==1]
    
    # Downsample majority class
    df_majority_downsampled = resample(df_majority,
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority), # to match minority class
                                     random_state=123) # reproducible results
    
    # Combine minority class with downsampled majority class
    df_balanced = pd.concat([df_majority_downsampled, df_minority])
    
    # Display new class counts
    print(df_balanced.death.value_counts())
    print([len(df_balanced.loc[df_balanced['death']==1]),len(df_balanced.loc[df_balanced['death']==0, 'SUBJECT_ID'].unique())])
    
    # Separate X_train and y_train after balancing
    X_train_balanced = df_balanced.drop('death', axis=1)
    y_train_balanced = df_balanced['death']
    X_train = X_train_balanced
    y_train = y_train_balanced
else:
    print("Nothing happened")

death
0    263
1    263
Name: count, dtype: int64
[263, 190]


In [82]:
#feature selection

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression  # Replace with an appropriate estimator
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.sparse import csr_matrix


if createDummies==1:

#X_train_sparse = csr_matrix(X_train)

    # Initialize your estimator 
    lr=LogisticRegression(penalty='l1',solver='liblinear')
    svc=SVC(kernel="linear")
    nb=GaussianNB()
    
    estimator = lr  #change to whatever works best
    
    # Initialize RFECV with the estimator and scoring method
    rfecv=RFECV(estimator=estimator, cv=5)
    kbest=SelectKBest(score_func=f_classif, k=1000)
    selector = kbest
    
    # Fit the selector on your training data
    X_train_selected = selector.fit_transform(X_train, y_train)
else:
    print("No feature selection, because of dimensionality reduction")

ValueError: k should be <= n_features = 8; got 1000. Use k='all' to return all features.

In [8]:

# Get selected feature indices
selected_feature_indices = selector.get_support(indices=True)

# Get the names of selected features
selected_feature_names = X_train.columns[selected_feature_indices]

# Create DataFrame with selected features
X_train_selected_df = X_train[selected_feature_names]

X_train_selected_df.columns
X_train_selected_df.head()

# no scaling necessary

In [72]:
from prince import MCA
#pip install prince

#applying MCA,PCA or others

#copy of categorical features
df_categorical = df.drop(columns=['TIME','time_since_last'], inplace=False)
def applyDimensionalityReduction(typeOfReduction,n_components):
    if typeOfReduction == 'mca':
        reducer = MCA(n_components=n_components)
        reducer.fit(df_categorical)
        transformed = mca.transform(df_categorical)
        df_mca = pd.concat([df[['TIME','time_since_last']],transformed],axis=1)
        return df_mca
    elif typeOfReduction =='pca':
        reducer = PCA(n_components=n_components)
        reducer.fit(df_categorical)
        transformed = mca.transform(df_categorical)
        return transformed
    
    else:
        return df

In [74]:
df = applyDimensionalityReduction('mca',2)
df.head()

KeyboardInterrupt: 

In [10]:
# do other funstuff